# <center> 【Kaggle】Telco Customer Churn 电信用户流失预测案例

&emsp;&emsp;<font face="仿宋">在案例的第二部分中，我们详细介绍了常用特征转化方法，其中有些是模型训练之必须，如自然数编码、独热编码，而有些方法则是以提高数据质量为核心、在大多数时候都是作为模型优化的备选方法，如连续变量分箱、数据标准化等。当然，在此之后，我们首先尝试构建一些可解释性较强的模型来进行用户流失预测，即采用逻辑回归和决策树模型来进行预测，并同时详细介绍了两种模型在实战中的调优技巧，在最终模型训练完成后，我们也重点讨论了关于两种可解释性模型建模结果的解释方法。

&emsp;&emsp;<font face="仿宋">从理论上来说，树模型的判别能力是要强于逻辑回归的，但在上一节最后的建模结果中我们发现两个模型的建模并无显著差别，预测准确率都维持在79%-80%之间，这或许说明很多逻辑回归无法正确判别的样本决策树模型也无法判别，据此我们推测，这是一个“入门容易、精通较难”的数据集。当然，如果我们进一步尝试其他“更强”的集成学习算法，如随机森林、XGB、CatBoost等，在当前数据集上的建模结果和逻辑回归也并无太大差异，因此我们亟需通过特征工程方法进一步提升数据集质量，进而提升最终模型效果。

&emsp;&emsp;<font face="仿宋">当然，哪怕是复杂模型在当前数据集上表现出了更好的效果，采用特征工程方法提升数据质量仍是优化建模结果必不可少的部分，正如时下流行的描述那样，“数据质量决定模型上界，而建模过程只是不断逼近这个上界”，特征工程中的一系列提高数据质量的方法、无论是在工业界实践中还是各大顶级竞赛里，都已然成了最为重要的提升模型效果的手段。

<center><img src="https://tva1.sinaimg.cn/large/008i3skNly1gwllgk4wgqj31hr0u0wh4.jpg" alt="image-20211112170651500" style="zoom:15%;" />

&emsp;&emsp;<font face="仿宋">不过，所谓的通过特征工程方法提高数据质量，看似简单但实际操作起来却并不容易。其难点并不在于其中具体操作方法的理解，至少相比机器学习算法原理，特征工程的很多方法并不复杂，特征工程的最大难点在于配合模型与数据进行方法选择、以及各种方法的工程化部署实现。一方面，特征工程方法众多，需要根据实际情况“因地制宜”，但数据的情况千变万化，很多时候需要同时结合数据探索结论、建模人员自身经验以及对各种备选方法的熟悉程度，才能快速制定行之有效的特征工程策略；另一方面，很多特征工程方法不像机器学习算法有现成的库可以直接调用，很多方法、尤其是一些围绕当前数据集的定制方法，需要自己手动实现，而这个过程就对建模人员本身的代码编写能力及工程部署能力提出了更高的要求。总而言之，特征工程是一个实践高度相关的技术，这也是为何课程会在介绍案例的过程中同步介绍特征工程常用方法的原因。

&emsp;&emsp;<font face="仿宋">当然，从宽泛的角度来看，所有围绕数据集的数据调整工作都可以看成是特征工程的一部分，包括此前介绍的缺失值填补、数据编码、特征变换等，这些方法其实都能一定程度提升数据质量，而本节开始，我们将花费一整节的时间来讨论另一类特征工程方法：特征衍生与特征筛选。而该方法通过创建更多特征来提供更多捕捉数据规律的维度，从而提升模型效果。当然特征衍生也是目前公认的最为有效的、能够显著提升数据集质量方法。

# <center>Part 4.时序特征衍生与NLP特征衍生

&emsp;&emsp;本阶开始我们将重点讨论特征工程中的特征衍生与特征筛选方法，并借此进一步提升模型效果。首先需要将此前的操作中涉及到的第三方库进行统一的导入：

In [1]:
# 基础数据科学运算库
import numpy as np
import pandas as pd

# 可视化库
import seaborn as sns
import matplotlib.pyplot as plt

# 时间模块
import time

# sklearn库
# 数据预处理
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer

# 实用函数
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

# 常用评估器
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

# 网格搜索
from sklearn.model_selection import GridSearchCV

# 自定义评估器支持模块
from sklearn.base import BaseEstimator, TransformerMixin

# 自定义模块
from telcoFunc import *

# re模块相关
import inspect, re

其中telcoFunc是自定义的模块，其内保存了此前自定义的函数和类，后续新增的函数和类也将逐步写入其中，telcoFunc.py文件随课件提供，需要将其放置于当前ipy文件同一文件夹内才能正常导入。

&emsp;&emsp;接下来导入数据并执行Part 1中的数据清洗步骤。

In [2]:
# 读取数据
tcc = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 标注连续/离散字段
# 离散字段
category_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
                'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                'PaymentMethod']

# 连续字段
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
 
# 标签
target = 'Churn'

# ID列
ID_col = 'customerID'

# 验证是否划分能完全
assert len(category_cols) + len(numeric_cols) + 2 == tcc.shape[1]

# 连续字段转化
tcc['TotalCharges']= tcc['TotalCharges'].apply(lambda x: x if x!= ' ' else np.nan).astype(float)
tcc['MonthlyCharges'] = tcc['MonthlyCharges'].astype(float)

# 缺失值填补
tcc['TotalCharges'] = tcc['TotalCharges'].fillna(0)

# 标签值手动转化 
tcc['Churn'].replace(to_replace='Yes', value=1, inplace=True)
tcc['Churn'].replace(to_replace='No',  value=0, inplace=True)

In [3]:
features = tcc.drop(columns=[ID_col, target]).copy()
labels = tcc['Churn'].copy()

接下来即可直接带入数据进行特征衍生。

#### 1.2 更通用的时序字段基本特征衍生方法

&emsp;&emsp;在本案例中，由于时序字段并没有包含更多细节的信息，因此分析过程并不复杂，但在其他很多场景下时序字段的特征衍生与分析可能会非常复杂。因此，为了应对更加复杂场景，我们需要进一步补充一些时序字段的处理方法与特征衍生策略，同时，在对某些包含时序特征的回归问题中，我们还会使用一类仅仅通过时序特征就能够对标签进行预测的模型——时间序列模型，尽管本案例中并不涉及实践序列模型建模，但简单了解其背后的核心思想却是非常有助于我们理解时序特征的重要性，也能够更进一步指导我们进行时序特征的特征衍生。

- Pandas中的时间记录格式

&emsp;&emsp;首先是关于时序字段的展示形式，本案例中tenure字段是经过自然数编码后的字段，时间是以整数形式进行呈现，而对于其他很多数据集，时序字段往往记录的就是时间是真实时间，并且是精确到年-月-日、甚至是小时-分钟-秒的字段，例如"2022-07-01;14:22:01"，此时拿到数据后，首先需要考虑的是如何对这类字段进行处理。

In [41]:
t = pd.DataFrame()

In [42]:
t['time'] = ['2022-01-03;02:31:52',
             '2022-07-01;14:22:01', 
             '2022-08-22;08:02:31', 
             '2022-04-30;11:41:31', 
             '2022-05-02;22:01:27']
t

,time
0,2022-01-03;02:31:52
1,2022-07-01;14:22:01
2,2022-08-22;08:02:31
3,2022-04-30;11:41:31
4,2022-05-02;22:01:27


当然，对于这类object对象，我们可以字符串分割的方式对其进行处理，此外还有一种更简单通用同时也更有效的方法：将其转化为datetime64格式，这也是pandas中专门用于记录时间对象的格式。对于datetime64来说有两种子类型，分别是datetime64[ns]毫秒格式与datetime64[D]日期格式。无论是哪种格式，我们可以通过pd.to_datetime函数对其进行转化，基本过程如下：

In [43]:
pd.to_datetime?

Signature:
pd.to_datetime(
    arg: Union[~DatetimeScalar, List, Tuple, ~ArrayLike, ForwardRef('Series')],
    errors: str = 'raise',
    dayfirst: bool = False,
    yearfirst: bool = False,
    utc: Union[bool, NoneType] = None,
    format: Union[str, NoneType] = None,
    exact: bool = True,
    unit: Union[str, NoneType] = None,
    infer_datetime_format: bool = False,
    origin='unix',
    cache: bool = True,
) -> Union[pandas.core.indexes.datetimes.DatetimeIndex, ForwardRef('Series'), ~DatetimeScalar, ForwardRef('NaTType')]
Docstring:
Convert argument to datetime.

Parameters
----------
arg : int, float, str, datetime, list, tuple, 1-d array, Series, DataFrame/dict-like
    The object to convert to a datetime.
errors : {'ignore', 'raise', 'coerce'}, default 'raise'
    - If 'raise', then invalid parsing will raise an exception.
    - If 'coerce', then invalid parsing will be set as NaT.
    - If 'ignore', then invalid parsing will return the input.
dayfirst : bool, default False


In [44]:
pd.to_datetime(t['time'])

0   2022-01-03 02:31:52
1   2022-07-01 14:22:01
2   2022-08-22 08:02:31
3   2022-04-30 11:41:31
4   2022-05-02 22:01:27
Name: time, dtype: datetime64[ns]

In [45]:
t['time'] = pd.to_datetime(t['time'])
t['time']

0   2022-01-03 02:31:52
1   2022-07-01 14:22:01
2   2022-08-22 08:02:31
3   2022-04-30 11:41:31
4   2022-05-02 22:01:27
Name: time, dtype: datetime64[ns]

> 这里需要注意，一般来说时间字段的记录格式都是用'-'来划分年月日，用':'来分割时分秒，用空格、分号或者换行来分割年月日与时分秒，这是一种通用的记录方法，如果是手动输入时间，也尽可能按照上述格式进行记录。

同时我们发现，该函数会自动将目标对象转化为datetime64[ns]类型，该对象类型是一种高精度、精确到纳秒（10的负九次方秒）的时间记录格式，后面在进行时间差计算时会看到纳秒级运算的结果。当然，如果我们的时间记录格式本身只精确到日期、并没有时分秒，通过pd.to_datetime函数仍然会转化为datetime64[ns]类型，只是此时显示的时间中就没有时分秒：

In [125]:
t1 = pd.DataFrame()
t1['time'] = ['2022-01-03', '2022-07-01',]
t1

,time
0,2022-01-03
1,2022-07-01


In [126]:
t1['time'] = pd.to_datetime(t1['time'])
t1['time']

0   2022-01-03
1   2022-07-01
Name: time, dtype: datetime64[ns]

就类似于浮点数与整数，更高的精度往往会导致更大的计算量，对于本身只是精确到日期的时间记录格式，我们其实可以用另一种只能精确到天的时间数据格式进行记录，也就是datetime64[D]类型。当然在pd.to_datetime函数使用过程中是无法直接创建datetime64[D]类型对象的，我们需要使用.value.astype('datetime64[D]')的方法对其进行转化，但是需要注意，这个过程最终创建的对象类型是array，而不再是Series了：

In [48]:
t1['time'].values

array(['2022-01-03T00:00:00.000000000', '2022-07-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [49]:
t1['time'].values.astype('datetime64[D]')

array(['2022-01-03', '2022-07-01'], dtype='datetime64[D]')

能够看出，array是支持datetime64[ns]和datetime64[D]等多种类型存储的，但对于pandas来说，只支持以datetime64[ns]类型进行存储，哪怕输入的对象类型是datetime64[D]，在转化为Series时仍然会被转化为datetime64[ns]：

In [50]:
t1['time'] = pd.Series(t1['time'].values.astype('datetime64[D]'), dtype='datetime64[D]')

In [51]:
t1['time']

0   2022-01-03
1   2022-07-01
Name: time, dtype: datetime64[ns]

尽管我们无法使用.values.astype('datetime64[D]')方法将Series对象类型进行'datetime64[D]'类型转化，但不同时间格式的转化有的时候有助于我们控制时间记录本身的精度，例如对于t中的time列，如果对其进行精确到天的'datetime64[D]'类型转化，则会自动删除时分秒的数据记录结果：

In [52]:
t['time'].values.astype('datetime64[D]')

array(['2022-01-03', '2022-07-01', '2022-08-22', '2022-04-30',
       '2022-05-02'], dtype='datetime64[D]')

In [53]:
t['time-D'] = t['time'].values.astype('datetime64[D]')
t

,time,time-D
0,2022-01-03 02:31:52,2022-01-03
1,2022-07-01 14:22:01,2022-07-01
2,2022-08-22 08:02:31,2022-08-22
3,2022-04-30 11:41:31,2022-04-30
4,2022-05-02 22:01:27,2022-05-02


有的时候我们无需考虑时分秒的时间时，可以通过上述方法进行精确到天的时间格式转化。当然，此外还有'datetime64[h]'类型可精确到小时、'datetime64[s]'精确到、'datetime64[ms]'精确到毫秒等：

In [54]:
t['time'].values.astype('datetime64[h]')

array(['2022-01-03T02', '2022-07-01T14', '2022-08-22T08', '2022-04-30T11',
       '2022-05-02T22'], dtype='datetime64[h]')

In [55]:
t['time'].values.astype('datetime64[s]')

array(['2022-01-03T02:31:52', '2022-07-01T14:22:01',
       '2022-08-22T08:02:31', '2022-04-30T11:41:31',
       '2022-05-02T22:01:27'], dtype='datetime64[s]')

> 在绝大多数情况下，我们都建议采用pandas中datetime类型记录时间（尽管从表面上来看也可以用字符串来表示时间），这也将极大程度方便我们后续对关键时间信息的提取。

- 时序字段的通用信息提取方式

&emsp;&emsp;在转化为datetime64格式之后，我们就可以通过一些dt.func方式来提取时间中的关键信息，如年、月、日、小时、季节、一年的第几周等，常用方法如下所示：

| 方法 | 作用 |
| ------ | ------ |
| dt.year | 提取年 |
| dt.month | 提取月 |
| dt.day | 提取日 |
| dt.hour | 提取小时 |
| dt.minute | 提取分钟 |
| dt.second | 提取秒 |

In [56]:
t['time'].dt.year

0    2022
1    2022
2    2022
3    2022
4    2022
Name: time, dtype: int64

In [57]:
t['time'].dt.second

0    52
1     1
2    31
3    31
4    27
Name: time, dtype: int64

接下来我们用不同的列记录这些具体时间信息：

In [58]:
t['year']        =  t['time'].dt.year
t['month']       =  t['time'].dt.month
t['day']         =  t['time'].dt.day
t['hour']        =  t['time'].dt.hour
t['minute']      =  t['time'].dt.minute
t['second']      =  t['time'].dt.second

In [59]:
t

,time,time-D,year,month,day,hour,minute,second
0,2022-01-03 02:31:52,2022-01-03,2022,1,3,2,31,52
1,2022-07-01 14:22:01,2022-07-01,2022,7,1,14,22,1
2,2022-08-22 08:02:31,2022-08-22,2022,8,22,8,2,31
3,2022-04-30 11:41:31,2022-04-30,2022,4,30,11,41,31
4,2022-05-02 22:01:27,2022-05-02,2022,5,2,22,1,27


- 时序字段的其他自然周期提取方式

&emsp;&emsp;当然，除了用不同的列记录时序字段的年月日、时分秒之外，我们知道，还有一些自然周期也会对结果预测有较大影响，如日期所在季度。这里需要注意的是，对于时序字段，往往我们会尽可能的对其进行自然周期的划分，然后在后续进行特征筛选时再对这些衍生字段进行筛选，对于此前的数据集，我们能够清晰的看到季度特征对标签的影响，而很多时候，除了季度，诸如全年的第几周、一周的第几天，甚至是日期是否在周末，具体事件的时间是在上午、下午还是在晚上等，都会对预测造成影响。对于这些自然周期提取方法，有些自然周期可以通过dt的方法自动计算，另外则需要手动进行计算。首先我们先看能够自动完成计算的自然周期：

| 方法 | 作用 |
| ------ | ------ |
| dt.quarter | 提取季度 |
| dt.weekofyear | 提取年当中的周数 |
| dt.dayofweek, dt.weekday | 提取周几 |

In [60]:
t

,time,time-D,year,month,day,hour,minute,second
0,2022-01-03 02:31:52,2022-01-03,2022,1,3,2,31,52
1,2022-07-01 14:22:01,2022-07-01,2022,7,1,14,22,1
2,2022-08-22 08:02:31,2022-08-22,2022,8,22,8,2,31
3,2022-04-30 11:41:31,2022-04-30,2022,4,30,11,41,31
4,2022-05-02 22:01:27,2022-05-02,2022,5,2,22,1,27


In [61]:
t['time'].dt.weekofyear

<ipython-input-61-40c69577c0b6>:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  t['time'].dt.weekofyear


0     1
1    26
2    34
3    17
4    18
Name: time, dtype: int64

In [62]:
t['time'].dt.quarter

0    1
1    3
2    3
3    2
4    2
Name: time, dtype: int64

In [63]:
t['time'].dt.dayofweek

0    0
1    4
2    0
3    5
4    0
Name: time, dtype: int64

这里需要注意，每周一是从0开始计数，这里我们可以手动+1，对其进行数值上的修改：

In [64]:
t['time'].dt.dayofweek + 1

0    1
1    5
2    1
3    6
4    1
Name: time, dtype: int64

接下来我们将其拼接到原始数据集中：

In [65]:
t['quarter']     =  t['time'].dt.quarter
t['weekofyear']  =  t['time'].dt.weekofyear
t['dayofweek']   =  t['time'].dt.dayofweek + 1

<ipython-input-65-d6fa2c370bb6>:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  t['weekofyear']  =  t['time'].dt.weekofyear


In [66]:
t

,time,time-D,year,month,day,hour,minute,second,quarter,weekofyear,dayofweek
0,2022-01-03 02:31:52,2022-01-03,2022,1,3,2,31,52,1,1,1
1,2022-07-01 14:22:01,2022-07-01,2022,7,1,14,22,1,3,26,5
2,2022-08-22 08:02:31,2022-08-22,2022,8,22,8,2,31,3,34,1
3,2022-04-30 11:41:31,2022-04-30,2022,4,30,11,41,31,2,17,6
4,2022-05-02 22:01:27,2022-05-02,2022,5,2,22,1,27,2,18,1


接下来继续创建是否是周末的标识字段：

In [67]:
(t['dayofweek'] > 5).astype(int) 

0    0
1    0
2    0
3    1
4    0
Name: dayofweek, dtype: int32

In [68]:
t['weekend'] = (t['dayofweek'] > 5).astype(int) 

进一步创建小时所属每一天的周期，凌晨、上午、下午、晚上，周期以6小时为划分依据：

In [69]:
(t['hour'] // 6).astype(int) 

0    0
1    2
2    1
3    1
4    3
Name: hour, dtype: int32

In [70]:
t['hour_section'] = (t['hour'] // 6).astype(int) 

其中，0代表凌晨、1代表上午、2代表下午、3代表晚上。接下来查看时间信息衍生后的数据表：

In [71]:
t

,time,time-D,year,month,day,hour,minute,second,quarter,weekofyear,dayofweek,weekend,hour_section
0,2022-01-03 02:31:52,2022-01-03,2022,1,3,2,31,52,1,1,1,0,0
1,2022-07-01 14:22:01,2022-07-01,2022,7,1,14,22,1,3,26,5,0,2
2,2022-08-22 08:02:31,2022-08-22,2022,8,22,8,2,31,3,34,1,0,1
3,2022-04-30 11:41:31,2022-04-30,2022,4,30,11,41,31,2,17,6,1,1
4,2022-05-02 22:01:27,2022-05-02,2022,5,2,22,1,27,2,18,1,0,3


至此，我们就完成了围绕时序字段的详细信息衍生（年月日、时分秒单独提取一列），以及基于自然周期划分的时序特征衍生（第几周、周几、是否是周末、一天中的时间段）。

<center><img src="https://s2.loli.net/2022/02/17/y7wctiB4EH3qkMV.png" alt="image-20220217202852046" style="zoom:33%;" />

&emsp;&emsp;当然整个时序字段的特征衍生过程并不复杂，接下来我们重点探讨为何我们需要对时序特征进行周期性划分，或者说，为何对时序特征进行有效的周期性划分是我们进一步挖掘时序特征背后有效信息的必要手段。

#### 2.2 NLP特征衍生的简单应用

&emsp;&emsp;正所谓“他山之石、可以攻玉”，很多时候NLP特征衍生能够在结构化数据处理中起到意想不到的效果。在了解了NLP中基本的CountVectorizer和TF-IDF方法之后，接下来进一步考虑如何将这些方法应用到结构化数据的特征衍生中。

##### 2.2.1 CountVectorizer与分组统计特征衍生

- 基本原理

&emsp;&emsp;先看一个最简单的应用场景，即CountVectorizer在分组统计特征衍生中的应用。我们先从一个例子入手进行观察特征衍生的过程，然后再讨论其背后的本质及原理。当然，在这个例子中，与其说是借助了CountVectorizer进行了特征衍生，不如说CountVectorizer和我们此前介绍的方法殊途同归。

&emsp;&emsp;在电信用户数据集中，存在一些彼此“类似”但又相互补充的离散字段，即用户购买服务字段，这些字段记录了用户此前购买的一系列服务，包括是否开通网络安全服务（OnlineSecurity）、是否开通在线备份服务（OnlineBackup）、是否开通设备安全服务（DeviceProtection）等，这些字段都是二分类字段，假设有如下极简数据集：

<center><img src="https://s2.loli.net/2022/02/23/qeXDcR7IOUhpojS.png" alt="image-20220223165413377" style="zoom: 50%;" />

&emsp;&emsp;接下来，我们将上述数据集类比于一个文本。这里首先将OnlineSecurity、OnlineBackup和DeviceProtection视作文本中的不同单词（Term），并根据tenure不同取值对用户进行分组，每个分组视作一个Document，则可将原数据集转化为文本数据，然后再使用CountVectorizer进行计算，该过程也非常简单，就是对每个文件进行词频的汇总，也就是分成两组后进行OnlineSecurity、OnlineBackup和DeviceProtection三个字段的汇总，基本过程如下：

<center><img src="https://s2.loli.net/2022/02/23/osv7ECDTV8zmNZX.png" alt="image-20220223175816675" style="zoom:50%;" />

最终，即可衍生出分组统计量OnlineSecurity_count、OnlineBackup_count和DeviceProtection_count。当然，这只是一个非常见的示例，并且如果从最后的结果上来看，其实就是分组求和的结果，但我们仍然需要熟悉这个先把结构化数据转化为文本数据、然后再使用NLP方法进行统计量计算的过程，以便于我们后续在这个过程中引入其他NLP方法以及进行其他形式的分组。

- 代码实现

&emsp;&emsp;具体的代码实现过程也非常简单，我们只需要对其进行分组求和集合，可以通过group过程快速实现。

&emsp;&emsp;首先先进行数据集的准备工作：

In [39]:
tar_col = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection']
keycol = 'tenure'
tar_col

['OnlineSecurity', 'OnlineBackup', 'DeviceProtection']

In [40]:
features[tar_col].head(5)

,OnlineSecurity,OnlineBackup,DeviceProtection
0,No,Yes,No
1,Yes,No,Yes
2,Yes,Yes,No
3,Yes,No,Yes
4,No,No,No


但根据此前介绍，这些特征其实都有三个取值：

In [41]:
features[tar_col].nunique()

OnlineSecurity      3
OnlineBackup        3
DeviceProtection    3
dtype: int64

In [42]:
features['OnlineSecurity'].explode().value_counts().to_dict()

{'No': 3498, 'Yes': 2019, 'No internet service': 1526}

我们需要将购买了服务标记为1，其他标记为0，可以通过如下操作实现：

In [43]:
(features['OnlineSecurity'] == 'Yes') * 1

0       0
1       1
2       1
3       1
4       0
       ..
7038    1
7039    0
7040    1
7041    0
7042    1
Name: OnlineSecurity, Length: 7043, dtype: int32

然后需要将这些列放到一个df中：

In [44]:
features_OE = pd.DataFrame()
features_OE[keycol] = features[keycol]

for col in tar_col:
    features_OE[col] = (features[col] == 'Yes') * 1

In [45]:
features_OE.head(5)

,tenure,OnlineSecurity,OnlineBackup,DeviceProtection
0,1,0,1,0
1,34,1,0,1
2,2,1,1,0
3,45,1,0,1
4,2,0,0,0


&emsp;&emsp;在准备好数据集之后，接下来使用groupby的方法、通过sum的方式计算组内求和结果：

In [46]:
count = features_OE.groupby(keycol).sum()
count

,OnlineSecurity,OnlineBackup,DeviceProtection
tenure,,,
0,4,4,4
1,37,47,41
2,27,38,30
3,20,35,33
4,24,32,26
...,...,...,...
68,51,65,58
69,38,50,49
70,66,77,78


整体过程并不复杂，和分组汇总统计特征一样，稍后我们将这些特征以tenure为主键拼接回原特征矩阵即可形成衍生特征。

##### 2.2.2 T-IDF与分组统计特征衍生

- 特征衍生过程

&emsp;&emsp;既然有CountVectorizer计算结果，自然而然我们就会联想到，接下来可以继续进行TF-IDF计算。该计算过程也并不复杂，直接在CountVectorizer结果上进行计算即可。对于上述极简示例，计算结果如下：

In [47]:
c2 = np.array([[1, 2, 2], [2, 1, 0]])
c2

array([[1, 2, 2],
       [2, 1, 0]])

In [48]:
transformer = TfidfTransformer(smooth_idf=True)

tfidf = transformer.fit_transform(c2)

tfidf.toarray()

array([[0.27840869, 0.55681737, 0.78258739],
       [0.89442719, 0.4472136 , 0.        ]])

<center><img src="https://s2.loli.net/2022/02/23/IWZx31Nnv2utKgR.png" alt="image-20220223201700208" style="zoom:50%;" />

据此又可以衍生出新的特征。当然，上述基于原数据集的count结果也可以进行TF-IDF计算：

In [49]:
transformer = TfidfTransformer(smooth_idf=True)

tfidf = transformer.fit_transform(count)

tfidf.toarray()[:5]

array([[0.57735027, 0.57735027, 0.57735027],
       [0.51021138, 0.64810634, 0.56536936],
       [0.48706002, 0.68549188, 0.5411778 ],
       [0.38390615, 0.67183577, 0.63344515],
       [0.50306617, 0.67075489, 0.54498835]])

&emsp;&emsp;据此，我们就又完成了基于TF-IDF的特征衍生。